In [ ]:
import pandas as pd
import numpy as np
#!pip install biopython
#from Bio.SeqUtils.ProtParam import ProteinAnalysis
#from Bio.SeqUtils import ProtParamData

## Input dataframe

## Sequence composition

In [ ]:
def sequence_composition(sequence:str)  -> np.ndarray[np.float64]:
  sequence = sequence[:40]
  alphabet = "AQLSREKTNGMWDHFYCIPV"
  composition_matrix = np.zeros((20,40))
  composition_vector = np.zeros((20))
  for i in range(len(alphabet)):
    for j in range(len(sequence)):
      if alphabet[i] == sequence[j]:
        composition_matrix[i][j] += 1
        composition_vector[i] += composition_matrix[i][j]
  composition_vector = composition_vector/40
  return composition_vector

print(sequence_composition("MPGKMVVILGASNILWIMFAASQAFKIETTPESRYLAQIGDSVSLTCSTTGCESPFFSWRTQIDSPLNGKVTNEGTTSTLTMNPVSFGNE"))

[0.125 0.05  0.075 0.075 0.025 0.05  0.05  0.05  0.025 0.075 0.075 0.025
 0.    0.    0.05  0.025 0.    0.125 0.05  0.05 ]


## Hydrophobicity and *a*I

In [ ]:
def hydrophobe(sequence: str, window_size: int) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
  kd_scale = {
    'R': -4.5,  # Arg
    'K': -3.9,  # Lys
    'N': -3.5,  # Asn
    'D': -3.5,  # Asp
    'Q': -3.5,  # Gln
    'E': -3.5,  # Glu
    'H': -3.2,  # His
    'P': -1.6,  # Pro
    'Y': -1.3,  # Tyr
    'W': -0.9,  # Trp
    'S': -0.8,  # Ser
    'T': -0.7,  # Thr
    'G': -0.4,  # Gly
    'A':  1.8,  # Ala
    'M':  1.9,  # Met
    'C':  2.5,  # Cys
    'F':  2.8,  # Phe
    'L':  3.8,  # Leu
    'V':  4.2,  # Val
    'I':  4.5   # Ile
    }

  a = 2.9 # relative volume of valine
  b = 3.9 # relative volume leucine/isoleucine


  sequence = sequence[:40]
  # padding to have as many scores as residues  (each window refers to the central res)
  d = int(window_size/2)
  sequence = "X"*d + sequence + "X"*d #padding
  hydrophobicities = np.array([])
  AIs = np.array([])
  for i in range(len(sequence)-(window_size)+1):
      counts_A = 0
      counts_V = 0
      counts_I = 0
      counts_L = 0
      hydrophobicity_score = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        if window[j] != "X":
          hydrophobicity_score = (hydrophobicity_score + kd_scale[window[j]])
          if window[j] == 'A':
              counts_A += 1
          elif window[j] == 'V':
              counts_V += 1
          elif window[j] == 'I':
              counts_I += 1
          elif window[j] == 'L':
                counts_L += 1
      X_A = (counts_A / window_size)
      X_V = (counts_V / window_size)
      X_I = (counts_I / window_size)
      X_L = (counts_L / window_size)
      AI = X_A + a * X_V + b * (X_I + X_L) #Aliphatic Index Formula
      hydrophobicities = np.append(hydrophobicities, hydrophobicity_score / window_size)
      AIs = np.append(AIs, AI)
  H_AI = np.array([hydrophobicities.mean(), hydrophobicities.max(), np.argmax(hydrophobicities), AIs.mean(), AIs.max(), np.argmax(AIs)])
  return hydrophobicities, AIs, H_AI
hydro_all, ai_all, feat = hydrophobe("MPGKMVVILGASNILWIMFAASQAFKIETTPESRYLAQIGDSVSLTCSTTGCESPFFSWRTQIDSPLNGKVTNEGTTSTLTMNPVSFGNE",5)
print("Hydrophobicity values along the windows:\n", hydro_all, "\n", len(hydro_all))
print("Aliphatic indexes along the windows:\n", ai_all, "\n", len(ai_all))
print("Final feature vector:\n", feat)

hydro_all, ai_all, feat = hydrophobe("MPGKMVVILGASNILWIMFAASQAFKIETTPESRYLAQIGDSVSLTCSTTGCESPFFSWRTQIDSPLNGKVTNEGTTSTLTMNPVSFGNE",7)
print("Hydrophobicity values along the windows:\n", hydro_all, "\n", len(hydro_all))
print("Aliphatic indexes along the windows:\n", ai_all, "\n", len(ai_all))
print("Final feature vector:\n", feat)


Hydrophobicity values along the windows:
 [-0.02 -0.8  -0.42  0.04  1.2   2.18  3.72  3.26  2.78  1.78  0.18  0.32
  1.16  0.62  1.68  2.76  2.42  2.02  2.56  1.5   0.42  0.22  0.42 -0.72
  0.34  0.34 -0.16 -0.86 -0.4  -2.   -1.46 -2.22 -2.34 -1.26 -0.2  -0.74
  1.06  1.24  0.48  0.12] 
 40
Aliphatic indexes along the windows:
 [0.   0.   0.   0.58 1.16 1.94 2.72 2.72 2.34 1.76 0.98 0.98 1.76 1.56
 2.34 2.34 1.56 0.98 1.18 0.4  0.4  0.6  0.4  0.2  0.98 0.98 0.78 0.78
 0.78 0.   0.   0.   0.   0.78 0.98 0.98 1.76 1.76 0.98 0.78] 
 40
Final feature vector:
 [0.5305 3.72   6.     1.0305 2.72   6.    ]
Hydrophobicity values along the windows:
 [-5.71428571e-01 -3.00000000e-01  3.00000000e-01  9.00000000e-01
  1.27142857e+00  2.04285714e+00  2.04285714e+00  2.85714286e+00
  2.47142857e+00  1.37142857e+00  1.41428571e+00  1.31428571e+00
  6.42857143e-01  1.34285714e+00  1.35714286e+00  1.87142857e+00
  2.62857143e+00  2.24285714e+00  1.58571429e+00  1.21428571e+00
  8.28571429e-01  9.5714285

# Aliphatic Index (combined)



In [ ]:
def aliphatic_index_windowed(sequence: str , window:int =9) -> np.ndarray[np.float64]:
    seq = seq.upper()
    L = len(seq)
    AI_values = []
    a = 2.9 # relative volume of valine
    b = 3.9 # relative volume leucine/isoleucine

    if L == 0 or L < window:
        return np.array([0.0])

    for i in range(L - window + 1):
        sub = seq[i:i+window]

        counts_A = 0
        counts_V = 0
        counts_I = 0
        counts_L = 0
        for aa in sub:
            if aa == 'A':
                counts_A += 1
            elif aa == 'V':
                counts_V += 1
            elif aa == 'I':
                counts_I += 1
            elif aa == 'L':
                counts_L += 1

        X_A = (counts_A / window) * 100
        X_V = (counts_V / window) * 100
        X_I = (counts_I / window) * 100
        X_L = (counts_L / window) * 100

        AI = X_A + a * X_V + b * (X_I + X_L) #Aliphatic Index Formula
        AI_values.append(AI)

    return np.array(AI_values)

# Secondary Structure Elements

In [ ]:
def SSE(sequence:str,window_size:int = 9)-> tuple [np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
  alpha_helix_scale = {
      "G": 0.570,  # Gly
      "P": 0.570,  # Pro
      "Y": 0.690,  # Tyr
      "C": 0.700,  # Cys
      "S": 0.770,  # Ser
      "T": 0.830,  # Thr
      "N": 0.670,  # Asn
      "R": 0.980,  # Arg
      "H": 1.000,  # His
      "D": 1.010,  # Asp
      "I": 1.080,  # Ile
      "W": 1.080,  # Trp
      "Q": 1.110,  # Gln
      "F": 1.130,  # Phe
      "K": 1.160,  # Lys
      "V": 1.060,  # Val
      "L": 1.210,  # Leu
      "A": 1.420,  # Ala
      "M": 1.450,  # Met
      "E": 1.510   # Glu
  }
  beta_sheet_scale = {
      "E": 0.370,  # Glu
      "D": 0.540,  # Asp
      "P": 0.550,  # Pro
      "G": 0.750,  # Gly
      "S": 0.750,  # Ser
      "K": 0.740,  # Lys
      "H": 0.870,  # His
      "N": 0.890,  # Asn
      "R": 0.930,  # Arg
      "A": 0.830,  # Ala
      "M": 1.050,  # Met
      "Q": 1.100,  # Gln
      "C": 1.190,  # Cys
      "T": 1.190,  # Thr
      "L": 1.300,  # Leu
      "F": 1.380,  # Phe
      "W": 1.370,  # Trp
      "Y": 1.470,  # Tyr
      "I": 1.600,  # Ile
      "V": 1.700   # Val
  }
  sequence = sequence[:40]
  # padding to have as many scores as residues (each window refers to the central res)
  d = int(window_size/2)
  sequence = "X"*d + sequence + "X"*d #padding
  alpha_helix = np.array([])
  beta_sheet = np.array([])
  for i in range(len(sequence)-(window_size)+1):
      alpha_score = 0
      beta_score = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        w = 1 - abs(j - (window_size - 1)/2) / ((window_size - 1)/2)
        if window[j] != "X":
          alpha_score = alpha_score + w * alpha_helix_scale[window[j]]
          beta_score = beta_score + w * beta_sheet_scale[window[j]]
      alpha_helix = np.append(alpha_helix, alpha_score / window_size)
      beta_sheet = np.append(beta_sheet, beta_score / window_size)
  alpha_feature = np.array([alpha_helix.mean(), alpha_helix.max(), np.argmax(alpha_helix)])
  beta_feature = np.array([beta_sheet.mean(), beta_sheet.max(), np.argmax(beta_sheet)])
  return alpha_helix, alpha_feature, beta_sheet, beta_feature

alpha_all, alpha_feat, beta_all, beta_feat = SSE("MPGKMVVILGASNILWIMFAASQAFKIETTPESRYLAQIGDSVSLTCSTTGCESPFFSWRTQIDSPLNGKVTNEGTTSTLTMNPVSFGNE",9)
print(alpha_all)
print(alpha_feat)
print(beta_all)
print(beta_feat)

[0.2725     0.33638889 0.39805556 0.4575     0.4825     0.50083333
 0.49194444 0.47944444 0.46055556 0.43333333 0.43388889 0.41861111
 0.42055556 0.44888889 0.47333333 0.50138889 0.53027778 0.55722222
 0.555      0.55444444 0.54444444 0.51694444 0.51916667 0.52111111
 0.52555556 0.52972222 0.51388889 0.48472222 0.45305556 0.42805556
 0.41638889 0.42222222 0.41972222 0.43694444 0.45361111 0.47777778
 0.4925     0.44972222 0.3725     0.25444444]
[ 0.46097917  0.55722222 17.        ]
[0.22472222 0.28138889 0.35472222 0.43361111 0.51583333 0.605
 0.63583333 0.61611111 0.54888889 0.46333333 0.42777778 0.42944444
 0.47194444 0.54555556 0.58027778 0.60416667 0.59583333 0.54638889
 0.50388889 0.45138889 0.41388889 0.41305556 0.42027778 0.44916667
 0.46527778 0.46083333 0.46916667 0.43444444 0.4125     0.38361111
 0.335      0.34111111 0.37305556 0.41944444 0.47777778 0.51416667
 0.50944444 0.47944444 0.41416667 0.30083333]
[0.45806944 0.63583333 6.        ]


## Charge

In [ ]:
# relating window charge to the context we get more variability across the sequence
def charge_seq(sequence: str, window_size:int = 3) -> tuple[np.ndarray, np.ndarray]:
  res_charges = {
        'K': 1,   # Lys
        'R': 1,   # Arg
        'H': 0.5, # His (partial positive)
        'D': -1,  # Asp
        'E': -1   # Glu
    }

  sequence = sequence[:20]
  # padding to have as many scores as residues  (each window refers to the central res)
  d = window_size//2
  sequence = "X"*d + sequence + "X"*d #padding

  norm_charges = np.array([])

  for i in range(len(sequence)-(window_size)+1):
      charge = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        if window[j] != "X" and window[j] in res_charges:
          charge = (charge + res_charges[window[j]])
      norm_charges = np.append(norm_charges, charge / window_size)
  charge_feature = np.array([norm_charges.max(), np.argmax(norm_charges), norm_charges.min(), np.argmin(norm_charges)])
  return norm_charges, charge_feature
norm, feat = charge_seq("MPGKMVVILGASNILWIMFAASQAFKIETTPESRYLAQIGDSVSLTCSTTGCESPFFSWRTQIDSPLNGKVTNEGTTSTLTMNPVSFGNE",3)
print("Normalized charge values along the windows:\n", norm, "\n", len(norm))
print("Final feature vector:\n", feat)

norm, feat = charge_seq("MKPPPRRRAAPARYLGEVTGPATWSAREKRQLVRLLQARQGQPEPDATELARELRGRSEAEIRVFLQQLKGRVAREAIQKVHPGGLQGPR",3)
print("Charge values along the windows:\n", norm, "\n", len(norm))
print("Final feature vector:\n", feat)

Normalized charge values along the windows:
 [0.         0.         0.33333333 0.33333333 0.33333333 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ] 
 20
Final feature vector:
 [0.33333333 2.         0.         0.        ]
Charge values along the windows:
 [ 0.33333333  0.33333333  0.33333333  0.          0.33333333  0.66666667
  1.          0.66666667  0.33333333  0.          0.          0.33333333
  0.33333333  0.33333333  0.         -0.33333333 -0.33333333 -0.33333333
  0.          0.        ] 
 20
Final feature vector:
 [ 1.          6.         -0.33333333 15.        ]
